In [16]:
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)
library(testthat)
library(digest)
library(tidymodels)

In [17]:
weather_data <- read.csv("PRSA_Data_Tiantan_20130301-20170228.csv")
weather_data

No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,2013,3,1,0,6,6,4,8,300,81,-0.5,1024.5,-21.4,0,NNW,5.7,Tiantan
2,2013,3,1,1,6,29,5,9,300,80,-0.7,1025.1,-22.1,0,NW,3.9,Tiantan
3,2013,3,1,2,6,6,4,12,300,75,-1.2,1025.3,-24.6,0,NNW,5.3,Tiantan
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
35062,2017,2,28,21,18,32,4,48,500,48,10.8,1014.2,-13.3,0,NW,1.1,Tiantan
35063,2017,2,28,22,15,42,5,52,600,44,10.5,1014.4,-12.9,0,NNW,1.2,Tiantan
35064,2017,2,28,23,15,50,5,68,700,21,8.6,1014.1,-15.9,0,NNE,1.3,Tiantan


In [18]:
tidy_data <- weather_data |>
            select(-No , -year, -month, -day, -hour, -PM10, -SO2, -NO2, -O3, -wd)

colnames(tidy_data) <-
            c('PM_2.5', 'CO', 'Temperature' , 'Pressure' , 'Dew_Point_Temperature' , 'Rain' , 'Wind_Speed' , 'Region')

tidy_data

PM_2.5,CO,Temperature,Pressure,Dew_Point_Temperature,Rain,Wind_Speed,Region
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
6,300,-0.5,1024.5,-21.4,0,5.7,Tiantan
6,300,-0.7,1025.1,-22.1,0,3.9,Tiantan
6,300,-1.2,1025.3,-24.6,0,5.3,Tiantan
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
18,500,10.8,1014.2,-13.3,0,1.1,Tiantan
15,600,10.5,1014.4,-12.9,0,1.2,Tiantan
15,700,8.6,1014.1,-15.9,0,1.3,Tiantan


In [21]:
weather_split <- initial_split(tidy_data, prop = 0.75, strata = "CO")
weather_training <- training(weather_split)
weather_testing <- testing(weather_split)
weather_training

,PM_2.5,CO,Temperature,Pressure,Dew_Point_Temperature,Rain,Wind_Speed,Region
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,6,300,-0.5,1024.5,-21.4,0,5.7,Tiantan
2,6,300,-0.7,1025.1,-22.1,0,3.9,Tiantan
3,6,300,-1.2,1025.3,-24.6,0,5.3,Tiantan
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
35029,109,2200,12.1,1017.3,-5.4,0,1.2,Tiantan
35030,118,2100,14.0,1015.8,-5.2,0,1.8,Tiantan
35058,9,NA,14.2,1012.5,-14.9,0,3.1,Tiantan
